In [11]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import ipaddress
from aimlsse_api.client import GroundDataClient, SatelliteDataClient
from shapely import Polygon

In [12]:
ground_data_client = GroundDataClient(ipaddress.ip_address('127.0.0.1'), 8000)

In [13]:
stations_meta = ground_data_client.queryMetadata(stations=['EDDF', 'EGLL'])
print(stations_meta)

                    geometry    id       name          plot_name   network  \
0   POINT (8.59861 50.04639)  EDDF  Frankfurt   FRANKFURT MAIN A  DE__ASOS   
1  POINT (-0.46144 51.47852)  EGLL     London  London - Heathrow  GB__ASOS   

    latitude  longitude   elevation  index_right ISO_A3_EH         country  \
0  50.046390    8.59861  113.000000           49       DEU         Germany   
1  51.478524   -0.46144   26.786364           78       GBR  United Kingdom   

  CONTINENT  distance_to_region  
0    Europe                 0.0  
1    Europe                 0.0  


In [37]:
stations_meta = ground_data_client.queryMetadata(polygons=[
    Polygon([(6.0, 47.5),(15.0, 47.5),(15.0, 55.0),(6.0, 55.0)])
])
print(stations_meta)

                      geometry    id                 name         plot_name  \
0     POINT (6.04222 54.03667)  EHHW          Buitengaats       Buitengaats   
1     POINT (6.58472 53.12361)  EHGG            Groningen  GRONINGEN/EELDE    
2    POINT (14.13806 47.53250)  LOXA  AIGEN/ENNSTAL (MIL)  AIGEN/ENNSTAL (M   
3    POINT (14.18751 48.23322)  LOWL                 Linz  LINZ (CIV/MIL)     
4    POINT (13.00000 47.80000)  LOWS             Salzburg  SALZBURG AIRPORT   
..                         ...   ...                  ...               ...   
108   POINT (7.51028 47.61417)  LFSB             Mulhouse  BALE/MULHOUSE      
109   POINT (6.22167 48.68778)  LFSN                Nancy  NANCY/ESSEY        
110   POINT (7.64028 48.54944)  LFST           Strasbourg  STRASBOURG/ENTZH   
111   POINT (6.21152 49.62658)  ELLX            Luxemburg  LUXEMBOURG (AUT)   
112  POINT (14.62278 53.39528)  EPSC             Szczecin  SZCZECIN/GOLENIO   

      network   latitude  longitude  elevation  ind

In [15]:
locations = stations_meta[['id', 'geometry']]
print(locations)

       id                   geometry
0    EHHW   POINT (6.04222 54.03667)
1    EHGG   POINT (6.58472 53.12361)
2    LOXA  POINT (14.13806 47.53250)
3    LOWL  POINT (14.18751 48.23322)
4    LOWS  POINT (13.00000 47.80000)
..    ...                        ...
108  LFSB   POINT (7.51028 47.61417)
109  LFSN   POINT (6.22167 48.68778)
110  LFST   POINT (7.64028 48.54944)
111  ELLX   POINT (6.21152 49.62658)
112  EPSC  POINT (14.62278 53.39528)

[113 rows x 2 columns]


In [16]:
satellite_data_client = SatelliteDataClient(ipaddress.ip_address('127.0.0.1'), 8010)

In [17]:
grid_cells = satellite_data_client.queryContainingGeometry(locations)
print(grid_cells)

                                             geometry   Name  \
0   GEOMETRYCOLLECTION Z (POLYGON Z ((5.67154 47.8...  31TGN   
1   GEOMETRYCOLLECTION Z (POLYGON Z ((4.56477 55.0...  31UFA   
2   GEOMETRYCOLLECTION Z (POLYGON Z ((4.53070 54.1...  31UFV   
3   GEOMETRYCOLLECTION Z (POLYGON Z ((5.71882 48.7...  31UGP   
4   GEOMETRYCOLLECTION Z (POLYGON Z ((5.76851 49.6...  31UGQ   
..                                                ...    ...   
57  GEOMETRYCOLLECTION Z (POLYGON Z ((13.63918 48....  33UVP   
58  GEOMETRYCOLLECTION Z (POLYGON Z ((13.58809 50....  33UVR   
59  GEOMETRYCOLLECTION Z (POLYGON Z ((13.56055 51....  33UVS   
60  GEOMETRYCOLLECTION Z (POLYGON Z ((13.50094 53....  33UVU   
61  GEOMETRYCOLLECTION Z (POLYGON Z ((13.46869 54....  33UVV   

                                          Description  
0   TILE PROPERTIES<br><table border=0 cellpadding...  
1   TILE PROPERTIES<br><table border=0 cellpadding...  
2   TILE PROPERTIES<br><table border=0 cellpadding...  
3   TIL

In [18]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [19]:
output_dir = 'results/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [35]:
markersize = 0.1

In [36]:
with plt.style.context(("seaborn", "ggplot")):
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    ax = world.plot(figsize=(16,9), color='#8e979f', zorder=1);
    grid_cells.buffer(0).boundary.plot(ax=ax, edgecolor='black', alpha=0.6, linewidth=0.1, zorder=2)
    ax.scatter(stations_meta.longitude, stations_meta.latitude, s=1.0*markersize, linewidth=0.05*markersize, color='#E74C3C', edgecolor='white', zorder=3)
    plt.savefig(f'{output_dir}locations_and_grid_cells_view.svg', bbox_inches='tight', dpi=150, format='svg')